# 目的

- nb001で売り買いできることがわかったから戦略を考える

- bitfrayerの取引データをどこかから探してくる！（ろうそく足データ）

https://blog.shoby.jp/entry/2017/05/24/093733#f-e6c220ff

- lightning fxでmmbotを運用する

In [5]:
import os 
import sys 
import time
import numpy as np
import pandas as pd
import talib

# 仮想通貨関連
import ccxt
import pybitflyer

# open high low close V

In [6]:
# ここのハードコーディング危険
api_key = '82BLyXP8xgK2JLjsKFMajV'
api_secret = 'uYu8qv+y9F7EBjzhfa2kitidpVfdyH4/riCopxQho+k='

--> lightning fxもできることがわかった！

In [7]:
bitflyer = ccxt.bitflyer({
'apiKey': api_key,
'secret': api_secret
})

In [8]:
# JPY残高を参照する関数
def get_asset():

    while True:
        try:
            value = bitflyer.fetch_balance()
            break
        except Exception as e:
            logger.info(e)
            time.sleep(1)
    return value

In [11]:
def get_colla():

    while True:
        try:
            value = bitflyer.privateGetGetcollateral()
            break
        except Exception as e:
            logger.info(e)
            time.sleep(1)
    return value

In [9]:
zandaka = get_asset()

In [12]:
syoko = get_colla()

In [21]:
OIN = 'BTC'
PAIR = 'BTCJPY28SEP2022'

In [26]:
value = bitflyer.fetchOrderBook('FX_BTC_JPY')

In [27]:
def get_status(id):

    if PAIR == 'BTC/JPY':
        PRODUCT = 'BTC_JPY'
    else:
        PRODUCT = PAIR

    while True:
        try:
            value = bitflyer.private_get_getchildorders(params = {'product_code': PRODUCT, 'child_order_acceptance_id': id})[0]
            break
        except Exception as e:
            logger.info(e)
            time.sleep(2)

    # APIで受け取った値を読み換える
    if value['child_order_state'] == 'ACTIVE':
        status = 'open'
    elif value['child_order_state'] == 'COMPLETED':
        status = 'closed'
    else:
        status = value['child_order_state']

    # 未約定量を計算する
    remaining = float(value['size']) - float(value['executed_size'])

    time.sleep(0.1)
    return {'id': value['child_order_acceptance_id'], 'status': status, 'filled': value['executed_size'], 'remaining': remaining, 'amount': value['size'], 'price': value['price']}

{'symbol': 'FX_BTC_JPY',
 'bids': [[5159377.0, 0.01000162],
  [5159203.0, 0.01],
  [5159067.0, 0.01],
  [5159066.0, 0.03477916],
  [5159042.0, 0.015],
  [5158867.0, 0.03477916],
  [5158861.0, 0.01],
  [5158814.0, 0.02],
  [5158782.0, 0.01],
  [5158771.0, 0.03476747],
  [5158761.0, 0.15],
  [5158719.0, 0.01],
  [5158713.0, 0.01],
  [5158705.0, 0.06851534],
  [5158704.0, 0.03512779],
  [5158620.0, 0.02],
  [5158545.0, 0.02],
  [5158472.0, 0.02],
  [5158471.0, 0.01],
  [5158467.0, 0.03476672],
  [5158460.0, 0.01],
  [5158453.0, 0.01],
  [5158397.0, 0.18],
  [5158355.0, 0.085],
  [5158354.0, 0.17056425],
  [5158328.0, 0.07345771],
  [5158303.0, 0.01],
  [5158284.0, 0.1],
  [5158202.0, 0.127],
  [5158083.0, 0.06786845],
  [5157798.0, 0.05],
  [5157686.0, 0.05],
  [5157677.0, 0.18],
  [5157676.0, 0.18],
  [5157606.0, 0.01],
  [5157429.0, 0.01],
  [5157422.0, 0.01],
  [5157293.0, 0.04991385],
  [5157207.0, 0.01],
  [5157194.0, 0.05],
  [5157183.0, 0.05],
  [5157178.0, 0.05],
  [5157057.0, 0.0

In [29]:
bitflyer.private_get_getchildorders(params = {'product_code': 'FX_BTC_JPY', 'child_order_acceptance_id': id})

[]

In [ ]:
tick = get_effective_tick(size_thru=AMOUNT_THRU, rate_ask=float(trade_ask['price']), size_ask=float(trade_ask['amount']), rate_bid=float(trade_bid['price']), size_bid=float(trade_bid['amount']))

In [30]:
size_thru = 0.01
rate_ask=0
size_ask=0
rate_bid=0
size_bid=0

In [35]:
while True:
        try:
            value = bitflyer.fetchOrderBook('FX_BTC_JPY')
            break
        except Exception as e:
            time.sleep(2)
i = 0
s = 0
while s <= size_thru:
    if value['bids'][i][0] == rate_bid:
        s += value['bids'][i][1] - size_bid
    else:
        s += value['bids'][i][1]
    i += 1

j = 0
t = 0
while t <= size_thru:
    if value['asks'][j][0] == rate_ask:
        t += value['asks'][j][1] - size_ask
    else:
        t += value['asks'][j][1]
    j += 1

time.sleep(0.5)

In [38]:
tick = {'bid': value['bids'][i-1][0], 'ask': value['asks'][j-1][0]}

In [39]:
ask = float(tick['ask'])
bid = float(tick['bid'])

In [40]:
ask, bid

(5170128.0, 5168407.0)

In [41]:
spread = (ask - bid) / bid

In [42]:
spread

0.00033298461208647074

In [43]:
# スプレッド閾値
SPREAD_ENTRY = 0.0005  # 実効スプレッド(100%=1,1%=0.01)がこの値を上回ったらエントリー
SPREAD_CANCEL = 0.0003 # 実効スプレッド(100%=1,1%=0.01)がこの値を下回ったら指値更新を停止

In [44]:
SPREAD_ENTRY, SPREAD_CANCEL

(0.0005, 0.0003)

In [47]:
DELTA = 1

In [45]:
trade_ask = limit('sell', amount_int_ask, ask - DELTA)

NameError: name 'limit' is not defined

In [48]:
ask-DELTA

5170127.0